## 실시간 객체 탐지 시스템 구현

- OpenCV, YOLO 활용한 실시간 객체 탐지 서비스 제작.

- 패키지 설치

    ```shell
    pip install pyqt5 cv2 ultralytics
    ```

1. __필수 라이브러리 임포트__

- YOLO : YOLOv8 객체 탐지 모델
- cv2 : OpenCV 라이브러리, 영상처리 및 웹캠 제어
- PyQt5 : GUI 생성, 웹캠 시작과 중지 버튼 추가

In [1]:
from ultralytics import YOLO
import cv2
from PyQt5.QtWidgets import QApplication, QLabel, QVBoxLayout, QWidget, QPushButton
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QImage, QPixmap

2. __VideoCaptureWidget 클래스 생성__

- 클래스 상속 : PyQt5의 QWidget을 상속받아 GUI창으로 활용
- 모델 로드 : YOLO  객체를 yolov8x.pt 모델 파일로 초기화

In [2]:
class VideoCaptureWidget(QWidget):
    def __init__(self):
        super().__init__()

        # YOLOv8x 모델 로드 (YOLOv8x)
        self.model = YOLO('yolov8x.pt')

         # 3. UI 설정
        self.setWindowTitle("실시간 객체 탐지") # 창 제목
        self.image_label = QLabel(self) # 객체 탐지된 웹캠 화면을 표시할 QLabel 설정
        self.layout = QVBoxLayout()
        self.layout.addWidget(self.image_label)

        self.start_button = QPushButton("Start Webcam", self)   # start 버튼 생성.
        self.start_button.clicked.connect(self.start_webcam)    # 클릭 시 5. star_webcam 호출
        self.layout.addWidget(self.start_button)

        self.stop_button = QPushButton("Stop Webcam", self) # stop 버튼 생성.
        self.stop_button.clicked.connect(self.stop_webcam)  # 클릭 시 stop_webcam 호출
        self.layout.addWidget(self.stop_button)
        self.setLayout(self.layout)

        # 4. 웹캠 초기화
        self.capture = None
        self.timer = QTimer(self)   # 타이머 설정. 주기적인 화면 업데이트
        self.timer.timeout.connect(self.update_frame)   # update_frame 함수 연결.

    # 5. star_webcam 함수 - 웹캠 시작
    def start_webcam(self):
        """웹캠을 시작하고, 타이머를 시작하여 프레임을 주기적으로 읽음"""
        self.capture = cv2.VideoCapture(0)  # 웹캠 장치 열기
        self.timer.start(20)  # 20ms마다 프레임 업데이트 (50fps)

    # 6. stop_webcam 함수 - 웹캠 중지
    def stop_webcam(self):
        """웹캠을 중지하고 타이머를 멈춤"""
        self.timer.stop()   # 프레임 업데이트 중지
        if self.capture is not None:
            self.capture.release()  # 웹캠 해제

    # 7. update_frame - 객체 탐지 및 UI 업데이트
    def update_frame(self):
        """웹캠에서 프레임을 읽어와서 YOLO 객체 탐지를 수행한 후 UI에 표시"""
        ret, frame = self.capture.read()    # 캠에서 프레임 읽어오기
        if ret:
            # YOLOv8 객체 탐지 수행
            results = self.model(frame)
            result = results[0]

            # 바운딩 박스가 포함된 이미지를 가져옴
            img_with_boxes = result.plot()

            # OpenCV 이미지를 QImage로 변환
            rgb_image = cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB)
            h, w, ch = rgb_image.shape
            bytes_per_line = ch * w
            convert_to_Qt_format = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)

            # QImage를 QLabel에 표시하기 위해 QPixmap으로 변환
            self.image_label.setPixmap(QPixmap.fromImage(convert_to_Qt_format))
    
    # 8. closeEvent - 프로그램 종료 시 자원 해제
    def closeEvent(self, event):
        """윈도우 닫을 때 웹캠 해제"""
        if self.capture is not None:
            self.capture.release()


9. __메인 실행부__

In [ ]:
if __name__ == "__main__":
    app = QApplication([])  # PyQt5 GUI 앱 실행. VideoCaptureWidget 객체를 띄움.
    window = VideoCaptureWidget()
    window.show()
    app.exec_() # 앱 종료. GUI의 이벤트 루프 실행.

### 결과

- 실행 화면  
![실행1](./app3-1.JPG)
- 웹캠 실행 후 탐지
![실행2](./app3-2.JPG)